# Выпускной проект. Работа с SQL.

## 1. Введение 

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## 2. Цели исследования

- Проанализировать базу данных с информацией о книгах, издательствах, авторах, а также о пользовательских обзорах книг, чтобы помочь сформулировать ценностное предложение для нового продукта.
- Для этого нам необходимо:
   - Посчитать, сколько книг вышло после 1 января 2000 года;
   - Для каждой книги посчитать количество обзоров и среднюю оценку;
   - Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — таким образом исключаем из анализа брошюры;
   - Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
   - Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## 3. Описание данных

**Таблица books**

Содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors**

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers**

Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

**Таблица ratings**

Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

**Таблица reviews**

Содержит данные о пользовательских обзорах на книги:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

## 4. Исследование базы данных

Импортируем библиотеки:

In [5]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

Устанавливаем параметры:

In [6]:
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db' # название базы данных
}

In [7]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

Сохраняем коннектор:

In [8]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### books

In [9]:
query_b = '''
    
    SELECT *,
        COUNT(*) OVER () AS table_size
    FROM books
    LIMIT 5
    
'''
pd.io.sql.read_sql(query_b, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id,table_size
0,1,546,'Salem's Lot,594,2005-11-01,93,1000
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336,1000
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135,1000
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309,1000
4,5,125,1776,386,2006-07-04,268,1000


Эта таблица содержит данные о книгах:

- book_id — идентификатор книги (первичный ключ);
- author_id — идентификатор автора (внешний ключ);
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя (внешний ключ).

Таблица состоит из 6 столбцов и 1000 строк.

### authors

In [10]:
query_a = '''
    
    SELECT *,
        COUNT(*) OVER () AS table_size
    FROM authors
    LIMIT 5
    
'''
pd.io.sql.read_sql(query_a, con = engine) 

,author_id,author,table_size
0,1,A.S. Byatt,636
1,2,Aesop/Laura Harris/Laura Gibbs,636
2,3,Agatha Christie,636
3,4,Alan Brennert,636
4,5,Alan Moore/David Lloyd,636


Таблица содержит данные об авторах:

- author_id — идентификатор автора (первичный ключ);
- author — имя автора.

Таблица состоит из 2 столбцов и 636 строк.

### publishers 

In [11]:
query_p = '''
    
    SELECT *,
        COUNT(*) OVER () AS table_size
    FROM publishers
    LIMIT 5
    
'''
pd.io.sql.read_sql(query_p, con = engine) 

,publisher_id,publisher,table_size
0,1,Ace,340
1,2,Ace Book,340
2,3,Ace Books,340
3,4,Ace Hardcover,340
4,5,Addison Wesley Publishing Company,340


Таблица содержит данные об издательствах:

- publisher_id — идентификатор издательства (первичный ключ);
- publisher — название издательства.

Объем таблицы: 340 строк.

### ratings

In [12]:
query_rat = '''
    
    SELECT *,
        COUNT(*) OVER () AS table_size
    FROM ratings
    LIMIT 5
    
'''
pd.io.sql.read_sql(query_rat, con = engine) 

,rating_id,book_id,username,rating,table_size
0,1,1,ryanfranco,4,6456
1,2,1,grantpatricia,2,6456
2,3,1,brandtandrea,5,6456
3,4,2,lorichen,3,6456
4,5,2,mariokeller,2,6456


В таблице находятся данные о пользовательских оценках книг:

- rating_id — идентификатор оценки (первичный ключ);
- book_id — идентификатор книги (внешний ключ);
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица состоит из 4 столбцов и 6456 записей.

### reviews

In [13]:
query_rev = '''
    
    SELECT *,
        COUNT(*) OVER () AS table_size
    FROM reviews
    LIMIT 5
    
'''
pd.io.sql.read_sql(query_rev, con = engine) 

,review_id,book_id,username,text,table_size
0,1,1,brandtandrea,Mention society tell send professor analysis. ...,2793
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...,2793
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...,2793
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...,2793
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...,2793


Таблица содержит данные о пользовательских обзорах на книги:

- review_id — идентификатор обзора (первичный ключ);
- book_id — идентификатор книги (внешний ключ);
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

Таблица состоит из 4 столбцов и 2793 строки

## 5. Написание запросов

### Задание 1

Посчитать, сколько книг вышло после 1 января 2000 года:

In [14]:
query_1 = '''
    
    SELECT
        COUNT(book_id)
    FROM books AS b
    WHERE publication_date > '2000-01-01'
    
'''
pd.io.sql.read_sql(query_1, con = engine) 

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Задание 2 

Для каждой книги посчитать количество обзоров и среднюю оценку:

In [15]:
query2 = ''' 

SELECT 
    books.title AS book_name,
    books.book_id AS id,
    SubRev.n_rev AS reviews,
    SubRat.avg_rat AS avg_rating
FROM books LEFT JOIN 
                (SELECT
                    book_id,
                    AVG(rating) AS avg_rat
                FROM
                    ratings
                GROUP BY 
                    book_id
                ) AS SubRat ON SubRat.book_id = books.book_id
        LEFT JOIN
                (SELECT 
                    book_id,
                    COUNT(review_id) AS n_rev
                FROM
                    reviews
                GROUP BY 
                    book_id
                ) AS SubRev ON SubRev.book_id = books.book_id
                
; ''' 

request2 = pd.io.sql.read_sql(query2, con = engine)
request2

,book_name,id,reviews,avg_rating
0,The Body in the Library (Miss Marple #3),652,2.0,4.500000
1,Galápagos,273,2.0,4.500000
2,A Tree Grows in Brooklyn,51,5.0,4.250000
3,Undaunted Courage: The Pioneering First Missio...,951,2.0,4.000000
4,The Prophet,839,4.0,4.285714
...,...,...,...,...
995,Alice in Wonderland,64,4.0,4.230769
996,A Woman of Substance (Emma Harte Saga #1),55,2.0,5.000000
997,Christine,148,3.0,3.428571
998,The Magicians' Guild (Black Magician Trilogy #1),790,2.0,3.500000


Получили таблицу с  заголовком, идентификатором, количеством обзоров и средним рейтингом книги.

### Задание 3

Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры:

In [16]:
query_3 = '''

WITH 
max AS (
    SELECT
        publisher_id,
        COUNT(book_id) AS book_cnt
    FROM books
    WHERE num_pages > 50
    GROUP BY publisher_id
    ORDER BY book_cnt DESC
    LIMIT 1
)

    SELECT
        publisher,
        book_cnt
    FROM publishers AS p
    JOIN max ON p.publisher_id = max.publisher_id
    
'''

pd.io.sql.read_sql(query_3, con = engine) 

,publisher,book_cnt
0,Penguin Books,42


Больше всего книг выпустило Penguin Books - 42 книги.

### Задание 4 

Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками:

In [17]:
query_4 = '''

WITH
a AS (   
    SELECT
        book_id
        , AVG(rating) AS book_rating
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating_id) >= 50
)
, c AS (   
    SELECT
        b.book_id
        , au.author
        , a.book_rating
    FROM books AS b
    JOIN a ON b.book_id = a.book_id
    JOIN authors AS au ON au.author_id = b.author_id
)

    SELECT
        author
        , AVG(book_rating) AS avg_rating
    FROM c
    GROUP BY author
    ORDER BY avg_rating DESC
    LIMIT 1

'''

pd.io.sql.read_sql(query_4, con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Среди популярных книг с количеством оценок равным или более 50, самый высокий рейтинг книг у J.K. Rowling/Mary GrandPré.

### Задание 5  

Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок:

In [18]:
query_5 = '''

WITH
a AS (    
    SELECT 
        username
    FROM ratings 
    GROUP BY username
    HAVING COUNT(rating_id) > 48
),    
b AS (    
    SELECT
        COUNT(review_id) AS review_cnt
    FROM reviews AS r
    JOIN a ON r.username = a.username
    GROUP BY a.username)
    
    SELECT
        AVG(review_cnt)
    FROM b
    
'''

pd.io.sql.read_sql(query_5, con = engine) 

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24,0.

## 6. Вывод 

**Исследование базы данных показало, что в базе данных содержатся 5 таблиц:**

- books - имеет 6 столбцов и 1000 записей;
- authors - имеет 2 поля и 656 записей;
- publishers - имеет 2 столбцa и 340 записей;
- ratings - имеет 4 столбцa и 6456 записей;
- reviews - имеет 4 поля и 2793 записей;

**Написание запросов дало следующие результаты:**

- после 1 января 2000 года вышло 819 книг;
- составили таблицу с заголовком, идентификатором, количеством обзоров и средним рейтингом книги;
- больше всего книг выпустило издательство Penguin Books - 42 книги;
- среди популярных книг с количеством оценок равным или более 50, самый высокий рейтинг книг у J.K. Rowling/Mary GrandPré.;
- среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24,0.